# Repair Prioritization Model (RPM)

In [2]:
import pandas as pd
import numpy as np
import cvxpy as cvx
import datetime

## Data

In [3]:
numSystems = 30

In [4]:
#list of systems that ned to repaired
systems = []
for i in range(0,numSystems):
    systems.append(''.join(['sys',str(i)]))

In [5]:
#spots in my line (equal to the number of systems that need to be repaired)
spots = [''.join(['spot',str(j)]) for j in range(0,len(systems))]

In [6]:
#cost of being placed in spot i
spot_cost = {}
for i in range(0,len(spots)):
    spot_cost[spots[i]] = i

In [7]:
#number of days each system has been offline
daysDeadlined = {}
for j in systems:
    daysDeadlined[j] = np.random.randint(low=0,high=21)

In [8]:
#if part is unavailable short term (expected sometime within the next n days) 
# -- currently assuming system only needs one part to be back online
short_unavail = {}
for j in systems:
    short_unavail[j] = np.random.randint(2)

In [9]:
#if part is unavailable long term (expected sometime beyond the next n days)
# -- currently assuming system only needs one part to be back online
long_unavail = {}
for j in systems:
    if short_unavail[j]==1:
        long_unavail[j] = np.random.randint(2)
    else:
        long_unavail[j] = 0

## Decisions

In [10]:
#whether system j gets assigned to spot i or not
x = {}
for i in spots:
    for j in systems:
        x[i,j] = cvx.Variable(boolean=True) 

## Objective

In [25]:
#minimize the total expected downtime (i.e. spot in line * to date days deadlined)
objective = cvx.Minimize(sum(spot_cost[i]*daysDeadlined[j]*x[i,j] for i in spots for j in systems))

## Constraints

In [26]:
#you can't assign more than one system to the same spot
c1 = [sum(x[i,j] for j in systems)==1 for i in spots]

#all systems must be assigned a spot
c2 = [sum(x[i,j] for i in spots) == 1 for j in systems]

c3 = []
spotsToSave = sum(short_unavail.values())
for j in systems:
    if short_unavail[j] == 0:
        for s in range(0,spotsToSave):
            c3.append(x[(''.join(['spot',str(s)])),j]==0)
            


constraints = c1 + c2 + c3

## Output

In [27]:
prob = cvx.Problem(objective,constraints)

In [29]:
prob.solve(solver='ECOS_BB')

3787.999999138458

In [30]:
print("status:", prob.status)
print("optimal value", int(abs(prob.value)))

status: optimal
optimal value 3787


In [31]:
decisions = pd.DataFrame(columns=['system','days deadlined','part available?'],index=spots)
for i in range(0,len(spots)):
    spot = ''.join(['spot',str(i)])
    for j in systems:
        var = x[spot,j].value
        if var > 0.5:
            decisions.loc[spot,'system'] = j
            decisions.loc[spot,'days deadlined'] = daysDeadlined[j]
            decisions.loc[spot,'part available?'] = short_unavail[j]
decisions

,system,days deadlined,part available?
spot0,sys0,19,1
spot1,sys20,18,1
spot2,sys9,18,1
spot3,sys8,17,1
spot4,sys27,17,1
spot5,sys18,15,1
spot6,sys22,14,1
spot7,sys10,13,1
spot8,sys13,13,1
spot9,sys26,12,1


# Speed testing

def RPM(numSystems):
    
    #list of systems that ned to repaired
    systems = []
    for i in range(0,numSystems):
        systems.append(''.join(['sys',str(i)]))
        
    #spots in my line (equal to the number of systems that need to be repaired)
    spots = [''.join(['spot',str(j)]) for j in range(0,len(systems))] 
    
    #cost of being placed in spot i
    spot_cost = {}
    for i in range(0,len(spots)):
        spot_cost[spots[i]] = i
        
    #number of days each system has been offline
    daysDeadlined = {}
    for j in systems:
        daysDeadlined[j] = np.random.randint(low=0,high=21)
        
    #if part is unavailable short term (expected sometime within the next n days) 
    # -- currently assuming system only needs one part to be back online
    short_unavail = {}
    for j in systems:
        short_unavail[j] = np.random.randint(2)
        
    #whether system j gets assigned to spot i or not
    x = {}
    for i in spots:
        for j in systems:
            x[i,j] = cvx.Variable(boolean=True)
            
    #minimize the total expected downtime (i.e. spot in line * to date days deadlined)
    objective = cvx.Minimize(sum(spot_cost[i]*daysDeadlined[j]*x[i,j] for i in spots for j in systems))
    
    #you can't assign more than one system to the same spot
    c1 = [sum(x[i,j] for j in systems)==1 for i in spots]

    #all systems must be assigned a spot
    c2 = [sum(x[i,j] for i in spots) == 1 for j in systems]

    c3 = []
    spotsToSave = sum(short_unavail.values())
    for j in systems:
        if short_unavail[j] == 0:
            for s in range(0,spotsToSave):
                c3.append(x[(''.join(['spot',str(s)])),j]==0)

    constraints = c1 + c2 + c3
    
    prob = cvx.Problem(objective,constraints)
    start = datetime.datetime.now()
    prob.solve(solver='ECOS_BB')
    end = datetime.datetime.now()
    timeElapsed = str(end - start)
    result = timeElapsed
    return result

numSystemsTest = [1,5,10,15,20,25,30,35,40,45,50,55,60,65,70,75]#,10,15,20,25,30,40,50,75]
rez = {}
for i in numSystemsTest:
    rez[i] = RPM(i)
rez